## Colab Specific Initialization

Colab initialization requires a kernel restart. So we do it at the beginning of the notebook.

In [1]:
import os

In [2]:
_is_colab = 'COLAB_GPU' in os.environ

Colab only allows one conda environment. We use `conda-colab` to install an anaconda style environment from a constructor style `.sh` file. See the `.environment` sub folder for instructions.

Colab kernel is restarted after the environment is installed.

In [4]:
if _is_colab:
    %pip install -q condacolab
    import condacolab
    condacolab.install_from_url("https://github.com/restlessronin/lila-reports/releases/download/v0.0.2/conda-lila-reports-0.0.2-Linux-x86_64.sh")

Since colab runtime is restarted by the previous cell, we need to reset globals

In [5]:
import os
_is_colab = 'COLAB_GPU' in os.environ

In colab we can mount data files directly from google drive.

In [ ]:
if _is_colab:
    from google.colab import drive
    drive.mount('/content/drive')

## Kaggle Specific Setup

Kaggle uses Dockerfiles to define environments, so we wind up using `pip` to install missing packages.

In [ ]:
_is_kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE','')
if _is_kaggle:
    %pip install -q matplotlib_scalebar

## Local Setup

For local notebooks, we use a conda environment created from `.environment/environment.yaml` to run the notebook. No package installation is necessary.

In [6]:
import sys, subprocess,time
import pandas as pd
import numpy as np
import geopandas as gpd
from pyproj import CRS
from shapely.ops import unary_union, polygonize
from rtree import index
from shapely.geometry import Polygon
from sklearn.cluster import DBSCAN
import rasterio
#import rasterstats as rs
import glob
from shapely.affinity import rotate
from shapely.geometry import LineString, Point, MultiPolygon,MultiLineString 
from shapely.ops import unary_union, polygonize
from geopandas import overlay
import shapely.wkt
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib_scalebar.scalebar import ScaleBar

from pathlib import Path

## Input and Working Dir paths

Since inputs data may be read only, we separate path abstractions for inputs and working directories. The path are wrapped in function calls, so this is the only cell which needs to be modified when the notebook runtime folders are changed.

In [7]:
def get_input(stem):
    return "../input/lilanagapattinam/LiLa_Nagapattinam/" + stem

def get_in_workdir(stem):
    if _is_kaggle or _is_colab:
        return './' + stem
    else:
        return get_input('workdir/' + stem)

def read_df_UT(stem):
    return gpd.read_file(get_input(stem)).to_crs(epsg = 4326)


Importing the powerline shape file (or geojson format)

In [8]:
df = read_df_UT("Supporting_info/osm_powline_11.776643009779821_10.743913945502888_80.19273383153288_79.14689901832789.geojson")

In [9]:
df.info()

In [10]:
df.plot(color = df["color"])
plt.show()

Importing the district shape file (Nagapattinam shape file)

In [11]:
df1 = read_df_UT("Practice/Nagapattinam_proj32644.shp")
df1.info()

In [12]:
df1.plot()

Merging the powerline shape and district shape file

In [13]:
merge = overlay(df, df1, how='intersection')
merge.info()

importing substation shape and overlaying with district shape file

In [14]:
df2 = read_df_UT("Supporting_info/list_substation_TN_corr.geojson/list_substation_TN_corr.shp")
intersection = overlay(df2, df1, how='intersection')

Plotting the powerline visuals with the help of the imported shape file and the base map is nagapattinam shape file.


In [15]:
ax = df1.plot(figsize=(5,5),color="none",zorder=2)
x = merge.plot(color="lightcoral",ax =ax) #powerline inside the district
y = intersection.plot(color="red",ax=ax)  #substation data points inside the district
ax.xaxis.tick_top() #making the x-axis scaling in the above
plt.xlim(79.40,80.00) #setting limit for x axis
plt.ylim(10.93,11.45) #setting limit for y axis 
plt.grid(color="grey",linestyle = '--', linewidth = 0.5) #creating the grid 
# plt.legend(loc='lower right') 
ax.tick_params(axis='x', colors='grey') #coloring the axis values 
ax.tick_params(axis='y', colors='grey')
# ax.add_artist(ScaleBar(1)) #adding the scale bar 
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f')) #axis value formatting for both axis
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.tick_params(axis='x', labelsize=5)  #reducing the size of the axis values
ax.tick_params(axis='y', labelsize=5)
# x1, y1, arrow_length = 0.9, 0.9, 0.1   #creating a north arrow
# ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
#             arrowprops=dict(facecolor='none', width=5, headwidth=12),
#             ha='center', va='center', fontsize=10,
#             xycoords=ax.transAxes)
ax.spines['bottom'].set_color('lightgrey') #changing the boundary colour
ax.spines['top'].set_color('lightgrey') 
ax.spines['right'].set_color('lightgrey')
ax.spines['left'].set_color('lightgrey')
plt.savefig(get_in_workdir("powerlines.jpg"),dpi =1500) #saving the picture in high quality
plt.show()

In [16]:
intersection.head()

Cutting the raster for a particular district

In [17]:
# data1 = "D:\\lila visuals\\DEM_T44PLT_proj32644_filled_slope.tif"
# data2 = "D:\\lila visuals\\DEM_T44PLT_proj32644_filled_slope - Copy (2).tif"

In [ ]:
## Colab Specific Initialization

In [18]:
# df = pd.DataFrame()   
# for j in range(len(df1)):  #df1 is nagapattinam district shape file 
#     input_shp = "D:\\lila visuals\\new.shp"

#     selection = df1.geometry[j:j+1]
#     if selection.geometry.is_empty.bool():
#         rasterarr = []
#     else:
#         selection.to_file(input_shp)
            
#         #first for LandUse raster 
#         input_raster= data1
    
#         output_raster = data2
            
#     ds = gdal.Warp(data2,
#                           data1,
#                           format = 'GTiff',
#                           cutlineDSName = input_shp,
#                           cropToCutline=True,
#                           )
#     ds = None
            
#     raster = gdal.Open(data2, gdal.GA_ReadOnly)
#     lcarr = raster.ReadAsArray()
#             #remove nodata values
#     lcarrorig = lcarr.reshape(lcarr.size,1)
#     lcarrND = lcarr[lcarr!=-9999]

#     if (np.size(lcarrND)==0):
#                df.at[j, "ToTVegPix"]=0
#                df.at[j, "ToTPix"]=0
#                df.at[j, "ToTVegAr%"]=0
       
               
#     else:    
                
                  
#         veg_sum = len(lcarrND[(lcarrND==3)])
               
                
               
#         df.at[j, "ToTVegPix"]=veg_sum
#         df.at[j, "ToTPix"]=np.size(lcarrorig)
#         df.at[j, "ToTVegAr%"]=100*veg_sum/(np.size(10))
                        
       
                   
# df1 = pd.concat([df1, df], axis = 1)
    
    
# geometry = df1["geometry"].astype(str).map(shapely.wkt.loads) 

# df1 = gpd.GeoDataFrame(df1, crs="EPSG:"+str(4326), geometry=geometry)
    
    
    
#     # df2 = df2.to_crs(epsg=4326)

importing the edges shape file (primary and secondary roads)

In [19]:
data = read_df_UT("Supporting_info/output_osmroad_edges_11.776643009779821_10.743913945502888_80.19273383153288_79.14689901832789.geojson/edges.shp")
data.plot()

In [20]:
data.info()

In [21]:
data.head()

In [22]:
data.shape

In [23]:
data["highway"].unique()

In [24]:
# data1=data.query("'highway' == 'secondary' and 'highway' == 'primary'")
# print(data.apply(lambda row: row[data['highway'].isin(['secondary','primary'])]))
# data1 = data.apply(lambda row: row[data['highway'].isin(['secondary','primary'])])

In [25]:
data1 = data.apply(lambda row: row[data['highway'].isin(['secondary'])])

In [26]:
data2 = data.apply(lambda row: row[data['highway'].isin(['primary'])])

In [27]:
data3 = read_df_UT("Supporting_info/railway/Railways.shp")
data3.plot()

overlaying the separated shape files with the district shape file

In [28]:
merge_data1 = overlay(data1,df1,how ="intersection")
merge_data2 = overlay(data2,df1,how ="intersection")
merge_data3 = overlay(data3,df1,how ="intersection")


In [29]:
merge_data1.plot()
merge_data2.plot()
merge_data3.plot()

In [30]:
ax = df1.plot(figsize=(5,5),color="none",zorder=3)
x =merge_data1.plot(color="lightcoral",label ="Secondary roads",ax =ax)
y = merge_data2.plot(color="red",label ="Primary roads",ax=ax)
z = merge_data3.plot(color="orange",label ="Railway roads",ax=ax)
# addlabels(x="primary",y ="secondary")
ax.xaxis.tick_top()
plt.xlim(79.40,80.00)
plt.ylim(10.93,11.45)
plt.grid(color="grey",linestyle = '--', linewidth = 0.5)
plt.legend(loc='lower right')
ax.tick_params(axis='x', colors='grey')
ax.tick_params(axis='y', colors='grey')
ax.add_artist(ScaleBar(10000,location='lower left',scale_loc='top'))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f')) #axis value formatting for both axis
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.tick_params(axis='x', labelsize=5)  #reducing the size of the axis values
ax.tick_params(axis='y', labelsize=5)
x1, y1, arrow_length = 0.9, 0.9, 0.1
ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
            arrowprops=dict(facecolor='none', width=5, headwidth=12),
            ha='center', va='center', fontsize=10,
            xycoords=ax.transAxes)
ax.spines['bottom'].set_color('lightgrey')
ax.spines['top'].set_color('lightgrey') 
ax.spines['right'].set_color('lightgrey')
ax.spines['left'].set_color('lightgrey')
plt.savefig(get_in_workdir("roadway.jpg"),dpi =1500)
plt.show()

importing the water shape file (overall TN) and overlaying with the district shape file

In [31]:
water = read_df_UT("Practice/water_TN.shp")
water_merge = overlay(water,df1,how ="intersection")

In [32]:
# from matplotlib.ticker import FormatStrFormatter

plotting the waterbodies shape files 

In [33]:
ax = df1.plot(figsize=(5,5),color="none",zorder=1)
x =water_merge.plot(color="lightcoral",ax =ax)
ax.xaxis.tick_top()
plt.xlim(79.40,80.00)
plt.ylim(10.93,11.45)
plt.grid(color="grey",linestyle = '--', linewidth = 0.50)
# plt.legend(loc='lower right')
# plt.ticklabel_format(axis="both")
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.tick_params(axis='x', colors='grey')
ax.tick_params(axis='y', colors='grey')
ax.add_artist(ScaleBar(1))
ax.tick_params(axis='x', labelsize=5)
ax.tick_params(axis='y', labelsize=5)
x1, y1, arrow_length = 0.5, 1.2, 0.09
ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
            arrowprops=dict(facecolor='none', width=3, headwidth=12),
            ha='center', va='center', fontsize=10,
            xycoords=ax.transAxes)
ax.spines['bottom'].set_color('lightgrey')
ax.spines['top'].set_color('lightgrey') 
ax.spines['right'].set_color('lightgrey')
ax.spines['left'].set_color('lightgrey')
plt.savefig(get_in_workdir("water.jpg"),dpi =1500)
plt.show()

In [34]:
def read_raster_UT_path(path):
    return rasterio.open(path)

def read_raster_UT(stem):
    return read_raster_UT_path(get_input(stem))

In [35]:
raster = read_raster_UT("Supporting_info/GHI_Nagapattinam.tif")

In [36]:
raster.meta

In [37]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [38]:
raster.crs

In [39]:
dstCrs = {'init': 'EPSG:4326'}

In [40]:
#calculate transform array and shape of reprojected raster
transform, width, height = calculate_default_transform(
        raster.crs, dstCrs, raster.width, raster.height, *raster.bounds)

raster.transform
transform

In [41]:
#working of the meta for the destination raster
kwargs = raster.meta.copy()
kwargs.update({
        'crs': dstCrs,
        'transform': transform,
        'width': width,
        'height': height,
    })

In [42]:
def open_raster_to_write(fname, kwargs):
    dstPath = get_in_workdir(fname)
    if os.path.exists(dstPath):
        os.remove(dstPath)
    return rasterio.open(dstPath, 'w', **kwargs)

dstRst = open_raster_to_write('GHIepsg_Nagapattinam.tif', kwargs)

In [43]:
dstRst.meta

In [44]:
#reproject and save raster band data
for i in range(1, raster.count + 1):
    reproject(
        source=rasterio.band(raster, i),
        destination=rasterio.band(dstRst, i),
        #src_transform=srcRst.transform,
        src_crs=raster.crs,
        #dst_transform=transform,
        dst_crs=dstCrs,
        resampling=Resampling.nearest)

#close destination raster
dstRst.close()        

In [45]:
import rasterio
from rasterio.plot import show
dstRst = read_raster_UT_path(get_in_workdir('GHIepsg_Nagapattinam.tif'))
show(dstRst)

land cover high, med, low

In [46]:
lc_high = read_df_UT('solar/_rl_elev_rd_wat_co_trans_ar_sub_rdpx_trsub_trat_subat_rdat_ir_high/LC_Solar_final_area_mask_1_Nagapattinam.shp')

In [47]:
lc_med = read_df_UT("solar/_rl_elev_rd_wat_co_trans_ar_sub_rdpx_trsub_trat_subat_rdat_ir_medatt/LC_Solar_final_area_mask_1_Nagapattinam.shp")

In [48]:
lc_low = read_df_UT('solar/_rl_elev_rd_wat_trans_ar_sub_rdpx_trsub_low/LC_Solar_final_area_mask_1_Nagapattinam.shp')

In [49]:
merge_lc1 = overlay(lc_high,df1,how ="intersection")
merge_lc2 = overlay(lc_med,df1,how ="intersection")
merge_lc3 = overlay(lc_low,df1,how ="intersection")


In [50]:
merge_lc3.head()

In [51]:
ax = df1.plot(figsize=(5,5),color="none",zorder=3)
x = merge_lc3.plot(color="yellow",ax =ax)
y = merge_lc2.plot(color="red",ax =ax)
z = merge_lc1.plot(color="green",ax =ax)
ax.xaxis.tick_top()
plt.xlim(79.40,80.00)
plt.ylim(10.93,11.45)
plt.grid(color="grey",linestyle = '--', linewidth = 0.50)
# plt.legend(loc="lower right")
plt.ticklabel_format(axis="both")
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.tick_params(axis='x', colors='grey')
ax.tick_params(axis='y', colors='grey')
ax.add_artist(ScaleBar(1))
ax.tick_params(axis='x', labelsize=5)
ax.tick_params(axis='y', labelsize=5)
x1, y1, arrow_length = 0.5, 1.2, 0.09
ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
            arrowprops=dict(facecolor='none', width=3, headwidth=12),
            ha='center', va='center', fontsize=10,
            xycoords=ax.transAxes)
ax.spines['bottom'].set_color('lightgrey')
ax.spines['top'].set_color('lightgrey') 
ax.spines['right'].set_color('lightgrey')
ax.spines['left'].set_color('lightgrey')
plt.savefig("lc.jpg",dpi =1500)
# plt.axis('off')
plt.show()

In [52]:
lc_tech = read_df_UT('solar/_rl_elev_rd_wat_co_trans_ar_sub_rdpx_trsub_tech/LC_Solar_final_area_mask_1_Nagapattinam.shp')

In [53]:
lc_theo = read_df_UT('solar/_rl_elev_rd_wat_co_th/LC_Solar_final_mask_val_1_Nagapattinam.shp')

In [54]:
lc_barren = read_df_UT('solar/all_lands_barren/all_BarrenLands_Mayu.shp')

In [55]:
lc_tech.plot()
lc_theo.plot()
lc_barren.plot()

In [56]:
ax = df1.plot(figsize=(5,5),color="none",zorder=3)
x = lc_barren.plot(color="yellow",ax =ax)
y = lc_theo.plot(color="red",ax =ax)
z = lc_tech.plot(color="green",ax =ax)
# ax.xaxis.tick_top()
plt.xlim(79.40,80.00)
plt.ylim(10.93,11.45)
# plt.grid(color="grey",linestyle = '--', linewidth = 0.50)
# # plt.legend(loc="lower right")
# plt.ticklabel_format(axis="both")
# ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# ax.tick_params(axis='x', colors='grey')
# ax.tick_params(axis='y', colors='grey')
# ax.add_artist(ScaleBar(1))
# ax.tick_params(axis='x', labelsize=5)
# ax.tick_params(axis='y', labelsize=5)
# x1, y1, arrow_length = 0.5, 1.2, 0.09
# ax.annotate('N', xy=(x1, y1), xytext=(x1, y1-arrow_length),
#             arrowprops=dict(facecolor='none', width=3, headwidth=12),
#             ha='center', va='center', fontsize=10,
#             xycoords=ax.transAxes)
# ax.spines['bottom'].set_color('lightgrey')
# ax.spines['top'].set_color('lightgrey') 
# ax.spines['right'].set_color('lightgrey')
# ax.spines['left'].set_color('lightgrey')
# plt.savefig("lc.jpg",dpi =1500)
plt.axis('off')
plt.show()

In [57]:
lc_theo = read_df_UT('solar/_rl_elev_rd_wat_co_th/LC_Solar_final_mask_val_1_Nagapattinam.shp')

In [58]:
lc_theo["area_class"].unique()

In [59]:
lc_theo_A = lc_theo[lc_theo["area_class"] == "A"]

In [60]:
lc_theo_A.head()

In [61]:
lc_theo_B = lc_theo[lc_theo["area_class"] == "B"]

In [62]:

lc_theo_C = lc_theo[lc_theo["area_class"] == "C"]

In [63]:
ax =df1.plot(df1,figsize=(5,5),color="none",zorder=3)
x = lc_theo_A.plot(color="black",ax =ax)
y = lc_theo_B.plot(color="green",ax =ax)
z = lc_theo_C.plot(color="red",ax =ax)
plt.axis('off')
plt.savefig(get_in_workdir("lc_a_b_c.jpg"),dpi =1500)
plt.show()